# Export risk and ambiguity data

This script outputs risk and ambiguity data from MAT files created by the aging scripts.

## load libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from glob import glob

Create a template for the Monetary task

In [2]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Risk_and_Ambiguity_new/AG_*_RA/MDM_MON_*.mat'
df = pd.DataFrame()
df_missing = pd.DataFrame()

The actual data extraction

In [3]:
for mat_fname in glob(glober):
    mat = sio.loadmat(mat_fname) # open the file
    sub = mat_fname.split('_')[5] # subject number
    
    # the mat file has a data stracture named Datamon we collect the values from different sublists
    vals = pd.DataFrame(mat['Datamon'][0][0][8], columns = ['value']) # values
    risk = pd.DataFrame(mat['Datamon'][0][0][9], columns = ['risk']) 
    ambiguity = pd.DataFrame(mat['Datamon'][0][0][10], columns = ['ambiguity'])
    
    # the selection data in the Datamon is divided into the 4 blocks.
    # in each block there are 21 trials.
    
    choice = []
    value = []
    for j in range(4):
        for i in range(21):    
            choice.append(mat['Datamon'][0][0][17][j][i][0][0])
            # This was just to make sure the values match. 
            if mat['Datamon'][0][0][17][j][i][2][0] == '$0':
                value.append(mat['Datamon'][0][0][17][j][i][3][0])
            else:
                value.append(mat['Datamon'][0][0][17][j][i][2][0])
    
    # create a db with the choices
    db = pd.DataFrame(list(zip(choice, value)),
               columns =['choice', 'value'])
    
    # fill the db with values
    db['value'] = vals
    db['risk'] = risk
    db['ambiguity'] = ambiguity
    db['sub'] = sub
    
    # replace values from the name to numbers
    db['choice'] = db.choice.replace(['Reference'], 0)
    db['choice'] = db.choice.replace(['Lottery'], 1)
    db['choice'] = db.choice.replace(['None'], np.nan)
    
    db['rt'] = pd.DataFrame(mat['Datamon']['rt'][0][0][0], columns = ['RT'])
    
    # look for catch trials if more than 5 (half) print sub number.
    catch = np.nansum(db.choice[db.value == 5])
    if catch > 5:
        print(sub, catch)
    #else:
    db['catch'] = catch
    #if db['choice'][db['choice'] != db['choice']].shape[0]:
    #    df_missing = df_missing.append(db)
    #else:
        # append the subject to the subjects dataframe
    df = pd.concat([df,db])

12 10
17 11
27 11
29 6
99 6.0
111 8


In [4]:
# save to csv
df.to_csv('data/mon.csv')
#df_missing.to_csv('mon_missing.csv')

Same idea only for the medical blocks.

In [5]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Risk_and_Ambiguity_new/AG_*_RA/MDM_MED_*.mat'
df = pd.DataFrame()
df_missing = pd.DataFrame()

In [6]:
for mat_fname in glob(glober):
    mat = sio.loadmat(mat_fname)
    
    sub = mat_fname.split('_')[5] # subject number
    vals = pd.DataFrame(mat['Datamed'][0][0][8], columns = ['value']) # values
    risk = pd.DataFrame(mat['Datamed'][0][0][9], columns = ['risk']) 
    ambiguity = pd.DataFrame(mat['Datamed'][0][0][10], columns = ['ambiguity'])
    
    # get data from all blocks
    choice = []
    value = []
    try:
        for j in range(4):
            for i in range(21):    
                choice.append(mat['Datamed'][0][0][17][j][i][0][0])

                if mat['Datamed'][0][0][17][j][i][2][0] == 'no effect':
                    value.append(mat['Datamed'][0][0][17][j][i][3][0])
                else:
                    value.append(mat['Datamed'][0][0][17][j][i][2][0])
    except:
        print("failed choice", sub)
        continue
    
    db = pd.DataFrame(list(zip(choice, value)),
               columns =['choice', 'value'])
    
    db['value'] = vals
    db['risk'] = risk
    db['ambiguity'] = ambiguity
    db['sub'] = sub
    
    db['choice'] = db.choice.replace(['Conservative treatment: Slight improvement'], 0)
    db['choice'] = db.choice.replace(['Experimental treatment'], 1)
    db['choice'] = db.choice.replace(['None'], np.nan)
    
    db['rt'] = pd.DataFrame(mat['Datamed']['rt'][0][0][0], columns = ['RT'])

    catch = np.nansum(db.choice[db.value == 5])
    if catch > 5:
        print(sub, catch)
    #else:
    db['catch'] = catch
    #if db['choice'][db['choice'] != db['choice']].shape[0]:
    #    df_missing = df_missing.append(db)
    #else:
    df = pd.concat([df,db])

22 8.0
12 12
17 6
failed choice 28
60 6
88 6.0
112 7
111 9


In [7]:
df.to_csv('data/med.csv')
#df_missing.to_csv('med_missing.csv')